In [39]:
import DataLoading
from torch.utils.data import DataLoader,TensorDataset
import numpy as np


import os 
import random

In [13]:
filenames = os.listdir(f"{folder}")
executions = [fn.split(per)[0].split(sc)[1][1:-1] for fn in filenames]

array(['Network_Existing_Generation_Full', 'Network_Full_Generation_Full',
       'PINT_Network_Line_In_Node_1_Node_2_cac1',
       'PINT_Network_Line_In_Node_1_Node_2_cac2',
       'PINT_Network_Line_In_Node_1_Node_2_cac3',
       'PINT_Network_Line_In_Node_1_Node_3_cac1',
       'PINT_Network_Line_In_Node_1_Node_3_cac2',
       'PINT_Network_Line_In_Node_1_Node_3_cac3',
       'PINT_Network_Line_In_Node_2_Node_3_cac1',
       'PINT_Network_Line_In_Node_2_Node_3_cac2',
       'PINT_Network_Line_In_Node_2_Node_3_cac3',
       'TOOT_Network_Line_In_Node_1_Node_2_cac1',
       'TOOT_Network_Line_In_Node_1_Node_2_cac2',
       'TOOT_Network_Line_In_Node_1_Node_2_cac3',
       'TOOT_Network_Line_In_Node_1_Node_3_cac1',
       'TOOT_Network_Line_In_Node_1_Node_3_cac2',
       'TOOT_Network_Line_In_Node_1_Node_3_cac3',
       'TOOT_Network_Line_In_Node_2_Node_3_cac1',
       'TOOT_Network_Line_In_Node_2_Node_3_cac2',
       'TOOT_Network_Line_In_Node_2_Node_3_cac3'], dtype='<U39')

In [5]:
sc = "sc01"
period = "2030"
case = "3-bus"

folder = f"../Data/{case}_DC_fy"

all_executions = DataLoading.list_executions(folder=folder, per=period, sc=sc)
executions_start = 0
executions_end = 3
executions = all_executions[executions_start:executions_end]
te_s = 0.3
val_s = 0.4
outp = "SystemCosts"

dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, executions, period, sc, ["PowerFlow"],outp)

dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter)


input_f_sc01_Network_Existing_Generation_Full_2030.csv
14
input_f_sc01_Network_Full_Generation_Full_2030.csv
14
input_f_sc01_PINT_Network_Line_In_Node_1_Node_2_cac1_2030.csv
14


In [6]:
def get_random_hours_slicer(nb_available,nb_selected,min_offset = 1,sorted = True):
    assert(nb_available>=nb_selected*min_offset)
    idx_l_size = 0
    index_list = []
    counter = 0
    while idx_l_size < nb_selected:
        r = random.randint(0, nb_available)
        after = list(range(r,r+min_offset))
        before = list(range(r-min_offset+1,r+1))
        if set(index_list).isdisjoint(before) and set(index_list).isdisjoint(after):
            idx_l_size += 1
            index_list.append(r)
            counter = 0 
        counter += 1
        assert(counter <1e7)
    if sorted:
        return np.sort(index_list)
    else:
        return index_list


In [34]:
def get_random_days_slicer(hours_available,nb_selected,hours_in_day=24,sorted = True):
    assert(hours_available>=nb_selected*hours_in_day)

    index_list = []
    for i in range(nb_selected):
        r = random.randint(0, hours_available)
        i = hours_in_day * round(r/hours_in_day)

        while i in index_list: 
            r = random.randint(0, hours_available)
            i = hours_in_day * round(r/hours_in_day)
        this_index_list = [i+j for j in range(hours_in_day)]
        index_list.append(this_index_list)
    if sorted: 
        return np.sort((np.array(index_list)).flatten())
    else: 
        return np.array(index_list).flatten()

In [165]:
def generate_one_week_per_month_indices(df,hours_in_day = 24,days_in_week=7):
    """
    Generate a list of indices, one from each month, representing one random week.

    Parameters:
        date_indices (pd.Index): A pandas datetime index.

    Returns:
        list of int: A list of indices, one from each month, representing one random week.
    """
    h_in_w = hours_in_day*days_in_week
    #First, convert the index of the input df to a proper DateTime index
    dt_index = pd.to_datetime(df.index, format='%m-%d %H:%M:%S%z',utc = True).tz_localize(None) + pd.DateOffset(hours=1)
    
    # Group the date indices by month
    grouped = df.groupby(dt_index.to_period('M'))

    # Initialize a list to store the selected indices
    selected_indices = []
    hours_before_month_started = 0
    # Select one random week index from each month
    for _, group in grouped:
        if len(group) >= h_in_w:
            random_week_index = random.randint(0, len(group) // h_in_w - 1)
            print(random_week_index)

            for ix in range(random_week_index * h_in_w,random_week_index * (h_in_w) + h_in_w):
                selected_indices.append(hours_before_month_started+ix)
        hours_before_month_started += len(group) 

    return selected_indices

# Example usage:
# date_indices = pd.date_range(start='2023-01-01', periods=365, freq='D')
# selected_indices = generate_one_week_per_month_indices(date_indices)
selected_indices = generate_one_week_per_month_indices(df)


# Print or use the list of selected indices
#print("Selected indices, one from each month, representing one random week:", selected_indices)

2
2
2
2
1
1
0
3
1
1
1
0
Selected indices, one from each month, representing one random week: [336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 

In [164]:
selected_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [167]:
df.iloc[selected_indices,:].index

Index(['01-15 00:00:00+01:00', '01-15 01:00:00+01:00', '01-15 02:00:00+01:00',
       '01-15 03:00:00+01:00', '01-15 04:00:00+01:00', '01-15 05:00:00+01:00',
       '01-15 06:00:00+01:00', '01-15 07:00:00+01:00', '01-15 08:00:00+01:00',
       '01-15 09:00:00+01:00',
       ...
       '12-07 14:00:00+01:00', '12-07 15:00:00+01:00', '12-07 16:00:00+01:00',
       '12-07 17:00:00+01:00', '12-07 18:00:00+01:00', '12-07 19:00:00+01:00',
       '12-07 20:00:00+01:00', '12-07 21:00:00+01:00', '12-07 22:00:00+01:00',
       '12-07 23:00:00+01:00'],
      dtype='object', name='LoadLevel', length=2016)

In [108]:
import pandas as pd

# Example DataFrame with an object dtype index
data = {'Values': [1, 2, 3, 4, 5]}
index = ['01-01 00:00:00+01:00', '01-01 01:00:00+01:00', '01-01 02:00:00+01:00',
         '01-01 03:00:00+01:00', '01-01 04:00:00+01:00']
df = pd.DataFrame(data, index=index)

# Convert the index to a DatetimeIndex while preserving timezone information
df.index = pd.to_datetime(df.index, format='%m-%d %H:%M:%S%z')

# Print the resulting DataFrame with a DatetimeIndex
print(df)
df.index

                           Values
1900-01-01 00:00:00+01:00       1
1900-01-01 01:00:00+01:00       2
1900-01-01 02:00:00+01:00       3
1900-01-01 03:00:00+01:00       4
1900-01-01 04:00:00+01:00       5


DatetimeIndex(['1900-01-01 00:00:00+01:00', '1900-01-01 01:00:00+01:00',
               '1900-01-01 02:00:00+01:00', '1900-01-01 03:00:00+01:00',
               '1900-01-01 04:00:00+01:00'],
              dtype='datetime64[ns, UTC+01:00]', freq=None)

In [138]:
df =dfs_in["PINT_Network_Line_In_Node_1_Node_2_cac1"]


DatetimeIndex(['1900-01-01 00:00:00', '1900-01-01 01:00:00',
               '1900-01-01 02:00:00', '1900-01-01 03:00:00',
               '1900-01-01 04:00:00', '1900-01-01 05:00:00',
               '1900-01-01 06:00:00', '1900-01-01 07:00:00',
               '1900-01-01 08:00:00', '1900-01-01 09:00:00',
               ...
               '1900-12-30 14:00:00', '1900-12-30 15:00:00',
               '1900-12-30 16:00:00', '1900-12-30 17:00:00',
               '1900-12-30 18:00:00', '1900-12-30 19:00:00',
               '1900-12-30 20:00:00', '1900-12-30 21:00:00',
               '1900-12-30 22:00:00', '1900-12-30 23:00:00'],
              dtype='datetime64[ns]', name='LoadLevel', length=8736, freq=None)

In [ ]:
def get_random_week_each_month_slicer(nb_weeks,nb_months,nb_hours_in_week):
    
    pass

In [35]:
nb_available = len(dfs_in["Network_Existing_Generation_Full"])
sel = DataLoading.get_random_hours_slicer(nb_available,20)
sel = get_random_days_slicer(nb_available,3)
#dfs_in["Network_Existing_Generation_Full"].iloc[sel,:]
sel


array([1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330,
       1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341,
       1342, 1343, 5856, 5857, 5858, 5859, 5860, 5861, 5862, 5863, 5864,
       5865, 5866, 5867, 5868, 5869, 5870, 5871, 5872, 5873, 5874, 5875,
       5876, 5877, 5878, 5879, 6000, 6001, 6002, 6003, 6004, 6005, 6006,
       6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017,
       6018, 6019, 6020, 6021, 6022, 6023])

In [ ]:
sc = "sc01"
period = "2030"
case = "RTS24"
#folder = "../Data/3-bus_AC_12w_ext_o_dummy_LCOE"
#folder = "../Data/9n_AC_12w_ext_o_dummy_LCOE"
folder = f"../Data/{case}_AC_12w_ext_o_dummy_LCOE"

all_executions = DataLoading.list_executions(folder=folder, per=period, sc=sc)
executions_start = 0
executions_end = 1
executions = all_executions[executions_start:executions_end]
te_s = 0.3
val_s = 0.4
outp = "SystemCosts"

exec_name = f"{case}_AC_12w_dummy_{te_s}_v{val_s}_PF_LCOE_{executions_start}_{executions_end}"

folder_to_save = f"{exec_name}"

#Load inputs and outputs in dataframes
dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, executions, period, sc, ["PowerFlow"],outp)
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)

# Convert to pytorch tensors
ts_in, ts_out, ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in, dfs_out, dfs_inter_j, executions, te_s, val_s)

#Create dataloaders
d_ft_in, d_ft_out, d_ft_inter,maxs = DataLoading.concat_and_normalize_ext_out(ts_in, ts_out, ts_inter, executions)

#Create TensorDatasets
train = TensorDataset(d_ft_in['train'].float(), d_ft_out['train'].float(), d_ft_inter['train'])
validation = TensorDataset(d_ft_in['val'].float(), d_ft_out['val'].float(), d_ft_inter['val'].float())